# Unit conversion in bluemira
The mechanics of the unit system in bluemira are fairly staight forward
It aims to provide a useful user interface to convert units
internally in most situations the units are up to the developer

In [1]:
from dataclasses import dataclass

from pint.errors import DimensionalityError

import bluemira.base.constants as const
from bluemira.base.parameter_frame import Parameter, ParameterFrame, parameter_frame

## Raw conversion

In [2]:
print(const.raw_uc(1, "um^3", "m^3"))
# gas flow rate conversion @OdegC
print(const.raw_uc(1, "mol/s", "Pa m^3/s"))
print(const.gas_flow_uc(1, "mol/s", "Pa m^3/s"))
# gas flow rate conversion @25degC
print(const.gas_flow_uc(1, "mol/s", "Pa m^3/s", gas_flow_temperature=298.15))
# boltzmann constant conversion
print(const.raw_uc(1, "eV", "K"))

9.999999999999999e-19
2271.0954641485578
2271.0954641485578
2478.9570296023885
11604.518121550082


## Raw Temperature conversion with checks

In [3]:
try:
    const.to_kelvin(-300)
except ValueError as v:
    print(v)

print(const.to_celsius(10, unit="rankine"))

Negative temperature in K specified.
-267.59444444444443


## Parameters and Units
First I make a small ParameterFrame

In [4]:
@dataclass
class MyParameterFrame(ParameterFrame):
    """A ParameterFrame"""

    A: Parameter[float]


# this works the same
@parameter_frame
class MyDecoratedFrame:
    """A ParameterFrame made with a decorator"""

    A: Parameter[float]


mypf = MyParameterFrame.from_dict({"A": {"value": 5, "unit": ""}})
mydecpf = MyDecoratedFrame.from_dict({"A": {"value": 5, "unit": ""}})

print(mypf)
print(mydecpf)
# Both frames equal
assert all([dparam == param for dparam, param in zip(mydecpf, mypf)])  # noqa: S101

╒════════╤═════════╤═══════════════╤══════════╤═══════════════╤═════════════╕
│ name   │   value │ unit          │ source   │ description   │ long_name   │
╞════════╪═════════╪═══════════════╪══════════╪═══════════════╪═════════════╡
│ A      │       5 │ dimensionless │ N/A      │ N/A           │ N/A         │
╘════════╧═════════╧═══════════════╧══════════╧═══════════════╧═════════════╛
╒════════╤═════════╤═══════════════╤══════════╤═══════════════╤═════════════╕
│ name   │   value │ unit          │ source   │ description   │ long_name   │
╞════════╪═════════╪═══════════════╪══════════╪═══════════════╪═════════════╡
│ A      │       5 │ dimensionless │ N/A      │ N/A           │ N/A         │
╘════════╧═════════╧═══════════════╧══════════╧═══════════════╧═════════════╛


Trying to set a unit with the wrong dimension

In [7]:
mypf2 = MyParameterFrame.from_dict({"A": {"value": 5, "unit": ""}})

In [8]:
mypf == mypf2

True

In [ ]:
mydiffval = MyDecoratedFrame.from_dict({"A": {"value": 6, "unit": "m"}})

try:
    mypf.update_from_frame(mydiffval)
except DimensionalityError as de:
    print(de)

Changing a value of a parameter with a compatible but different unit

In [ ]:
mypf.update_values({"A": {"value": 6, "unit": ""}})

print(mypf)